In [ ]:
!pip install -Uqqq  pip --progress-bar off --root-user-action=ignore
!pip install -qqq torch --progress-bar off --root-user-action=ignore
!pip install -qqq -U transformers --progress-bar off --root-user-action=ignore
!pip install -qqq -U accelerate --progress-bar off --root-user-action=ignore
!pip install -qqq huggingface_hub --progress-bar off --root-user-action=ignore

In [ ]:
!pip install -qqq bitsandbytes --progress-bar off --root-user-action=ignore

In [ ]:
import torch
import transformers
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
    TextStreamer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)

import bitsandbytes as bnb

In [ ]:
torch.set_default_device('cuda')

In [ ]:
torch.set_default_device('cuda')


bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )


CogniAsess = AutoModelForCausalLM.from_pretrained(
    "Usaid/ContextClassy-Merged-V1",
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Usaid/ContextClassy-Merged-V1")
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/510 [00:00<?, ?B/s]

In [ ]:
OneShotExample = """
{
  "Difficulty Level": "Selected Difficulty",
  "Topic": "Selected Topic",
  "questions": [
    {
      "Question Text": "Text of Question 1",
    }
  ]
}
"""


SystemRole = """
Your name is ContextClassy. You are an advanced AI system designed to assess programming fundamentals skills across a range of topics with
high precision and adaptability. Your capabilities include generating difficulty level specific questions and quizzes to accurately
gauge an individual's skills and potential in their respective skill level. You are equipped with a comprehensive understanding
of various programming fundamentals topics, allowing you to create realistic scenarios and questions that challenge and measure the
abilities of candidates effectively. This enables you to generate assessments that are both challenging and relevant,
offering realistic insights into how individuals perform on problem solving questions. Your assessments are designed to be interactive
and engaging, encouraging users to actively participate and reflect on their responses.

You are currently tasked with creating questions for a user who has pre-selected a specific difficulty level and a corresponding
topic for practice. Your unique algorithmic design is focused on producing questions that are not only relevant to the chosen difficulty level
and topic but also provide depth and insight into the user's problem solving capabilities.
"""


def UserPrompt(Difficulty_Level , Topic):
  UserQuery = f"""
  Selected Difficulty Level: {Difficulty_Level}
  Selected Domain: {Topic}

  Your operational directives are as follows:

  Formulate questions that are directly relevant to the selected difficulty level and topic. These questions should reflect
  problem solving skills and challenges pertinent to the difficulty level, enabling the user to demonstrate their competency
  in the specified topic. Ensure that each question is designed to probe in-depth into the user's understanding,
  skills, and application in the topic. Your questions should not be generic but rather specific to the nuances
  and complexities of the topic and difficulty level selected. All questions should align with academic standards and best
  practices related to the selected topic. They should be structured to reflect the expectations and requirements
  of a problem solver operating in that topic.

  Your output must be formatted as follows:

  Present the generated questions in a JSON format only with one question only.
  This should include an array of object representing a question. The object
  must contain a key-value pairs: one for the question text and another
  for the question ID. The question should be clearly articulated, focusing specifically
  on the difficulty level and topic selected. They should be structured to challenge the user's knowledge
  and skills relevant to the topic.

  Here is an example of how your JSON output might look: {OneShotExample}

  """
  return UserQuery

def configure_bits_and_bytes():
    """
    Configures the BitsAndBytes settings for model quantization. This setup is essential
    for loading the model in a memory-efficient 4-bit format and specifying the
    quantization type and computation data type.

    :return: A configured BitsAndBytesConfig object.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    return bnb_config


In [ ]:
def stream_role_domain_prompt(user_prompt):
    runtimeFlag = "cuda:0"
    system_prompt = SystemRole
    E_INST = "</s>"
    user, assistant = "<|user|>" , "<|assistant|>"

    prompt = f"{system_prompt}{E_INST}\n{user}\n{user_prompt.strip()}{E_INST}\n{assistant}\n"

    inputs = tokenizer([prompt], return_tensors="pt").to(runtimeFlag)

    streamer = TextStreamer(tokenizer,  skip_prompt=True, skip_special_tokens=True)

    _ = CogniAsess.generate(**inputs, streamer=streamer, max_new_tokens=500)

def stream_personality_prompt(user_prompt):
    runtimeFlag = "cuda:0"
    system_prompt = SystemRole
    E_INST = "</s>"
    user, assistant = "<|user|>" , "<|assistant|>"

    prompt = f"{system_prompt}{E_INST}\n{user}\n{user_prompt.strip()}{E_INST}\n{assistant}\n"

    inputs = tokenizer([prompt], return_tensors="pt").to(runtimeFlag)

    streamer = TextStreamer(tokenizer,  skip_prompt=True, skip_special_tokens=True)

    _ = CogniAsess.generate(**inputs, streamer=streamer, max_new_tokens=500)

def stream_general_prompt(user_prompt):
    runtimeFlag = "cuda:0"
    system_prompt = SystemRole
    E_INST = "</s>"
    user, assistant = "<|user|>" , "<|assistant|>"

    prompt = f"{system_prompt}{E_INST}\n{user}\n{user_prompt.strip()}{E_INST}\n{assistant}\n"

    inputs = tokenizer([prompt], return_tensors="pt").to(runtimeFlag)

    streamer = TextStreamer(tokenizer,  skip_prompt=True, skip_special_tokens=True)

    _ = CogniAsess.generate(**inputs, streamer=streamer, max_new_tokens=500)

In [ ]:
stream_role_domain_prompt(UserPrompt("Hard", "Array"))

{
 "Difficulty Level": "Hard",
 "Topic": "Array",
 "questions": [
   {
     "Question ID": 1,
     "Question Text": "Given the array [5, 10, 15, 20], write a function in JavaScript that finds the maximum element in the array using recursion."
   }
 ]
}
